In [1]:
!pip3 install beautifulsoup4
!pip install Selenium
!pip install webdriver-manager
!pip install html_table_parser
!pip install chromedriver-autoinstaller
!pip install --upgrade beautifulsoup4
!pip install --upgrade html5lib

  Using cached beautifulsoup4-4.4.1-py3-none-any.whl (81 kB)
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.10.0
    Uninstalling beautifulsoup4-4.10.0:
      Successfully uninstalled beautifulsoup4-4.10.0
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.4.1
    Uninstalling beautifulsoup4-4.4.1:
      Successfully uninstalled beautifulsoup4-4.4.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
html-table-parser 0.1.0 requires beautifulsoup4==4.4.1, but you have beautifulsoup4 4.10.0 which is incompatible.


In [2]:
from selenium import webdriver 
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests
from html_table_parser import parser_functions as parser
from urllib.request import urlopen
from time import sleep


import chromedriver_autoinstaller


* 드라이버 가져오기

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
sleep(5)
url = 'https://bd.kma.go.kr/kma2020/fs/energySelect1.do?pageNum=5&menuCd=F050701000'
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\hongr\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


## 크롤링 항목

* 오늘(날짜)
* 각 시간에 대한 정보, 누적발전량, 일사량
* 전국에 대한 1차 수집

In [4]:
today = driver.find_element_by_id("thToday").text
print(today)

오늘(10/09)


In [5]:
area_name = driver.find_element_by_id("areaName").text
print(area_name)

전국


In [6]:
element = driver.find_element_by_xpath('//*[@id="toEnergy"]/tr[1]').text.split()
sleep(5)
print("시간당 출력되는 정보:",element)

시간당 출력되는 정보: ['0시', '0.00', '0.00', '0.0', '18.5', '3.6', '0.00', '0.00', '0.0', '19.2', '3.3']


In [7]:
time = []
elect = []
sun = []
for i in range(1,24):
    
    tr = str(i)
    string = driver.find_element_by_xpath(f'//*[@id="toEnergy"]/tr[{i}]').text.split()
    time.append(string[0])
    elect.append(string[2])
    sun.append(string[3])

print('time : ',time)
print('elect : ',elect)
print('sun : ',sun)

time :  ['0시', '1시', '2시', '3시', '4시', '5시', '6시', '7시', '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시', '21시', '22시']
elect :  ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '2.33', '205.01', '661.18', '1403.53', '2348.45', '3408.81', '4543.35', '5645.77', '6616.11', '7348.42', '7793.19', '7932.51', '7932.51', '7932.51', '7932.51', '7932.51']
sun :  ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '1.4', '6.3', '23.9', '83.5', '124.9', '142.2', '87.3', '48.8', '39.7', '70.6', '12.5', '2.2', '0.0', '0.0', '0.0', '0.0']


In [8]:
my_dict = {"누적발전량":elect,"일사량":sun}
my_df = pd.DataFrame(my_dict,index=time).rename_axis(f"전국:{today}")
display(my_df)

,누적발전량,일사량
전국:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,2.33,1.4
8시,205.01,6.3


## 각 지역별로 시간대별 누적발전량, 일사량 정보를 수집

*  서울특별시
*  경기도
*  인천광역시
*  대전광역시
*  세종특별자치시
*  광주광역시
*  울산광역시
*  부산광역시
*  제주특별자치도
*  강원도
*  충청북도
*  충청남도
*  경상북도
*  경상남도
*  전라북도
*  전라남도


## 각 지역별로 크롤링 전, 리스트에 담고, 반복문을 돌아 크롤링.

### 1. 셀레니움을 이용해 각 지역을 클릭

* 필요한 변수 : 오늘날짜를 담을 변수, 

### 2. 각 지역을 클릭시 생성된 DataFrame을 위에서 구축한 함수로 변수에 저장

 
### 3. 출력

* today = driver.find_element_by_id("thToday").text       == 오늘 정보
* area_name = driver.find_element_by_id("areaName").text  == 지역 이름
* driver.find_element_by_xpath('//*[@id="info_4200000000"]/span').click()    == 지역 클릭

In [9]:
서울특별시     = '//*[@id="info_1100000000"]/span' 
경기도         = '//*[@id="info_4100000000"]/span'
인천광역시     = '//*[@id="info_2800000000"]/span'
대전광역시     = '//*[@id="info_3000000000"]/span'
세종특별자치시 = '//*[@id="info_3600000000"]/span'
광주광역시     = '//*[@id="info_2900000000"]/span'
대구광역시     = '//*[@id="info_2700000000"]/span'
울산광역시     = '//*[@id="info_3100000000"]/span'
부산광역시     = '//*[@id="info_2600000000"]/span'
제주특별자치도 = '//*[@id="info_5000000000"]/span'
강원도         = '//*[@id="info_4200000000"]/span'
충청북도       = '//*[@id="info_4300000000"]/span'
충청남도       = '//*[@id="info_4400000000"]/span'
경상북도       = '//*[@id="info_4700000000"]/span'
경상남도       = '//*[@id="info_4800000000"]/span'
전라북도       = '//*[@id="info_4500000000"]/span'
전라남도       = '//*[@id="info_4600000000"]/span'

In [10]:
place = [서울특별시   
,경기도     
,인천광역시   
,대전광역시   
,세종특별자치시 
,광주광역시
,대구광역시         
,울산광역시   
,부산광역시   
,제주특별자치도 
,강원도     
,충청북도    
,충청남도    
,경상북도    
,경상남도    
,전라북도    
,전라남도    ]

* inforamtion(지역) ; 지역에 대한 dataframe 반환 변수

In [11]:
def information(area):
    
    button = driver.find_element_by_xpath(area)
    driver.execute_script("arguments[0].click();", button)
    
        
    time = []
    elect = []
    sun = []
    for i in range(1,24):

        tr = str(i)
        string = driver.find_element_by_xpath(f'//*[@id="toEnergy"]/tr[{i}]').text.split()
        time.append(string[0])
        elect.append(string[2])
        sun.append(string[3])
    
    
    today = driver.find_element_by_id("thToday").text       #오늘 정보
    area_name = driver.find_element_by_id("areaName").text  #지역 이름

    my_dict = {"누적발전량":elect,"일사량":sun}
    area_my_df = pd.DataFrame(my_dict,index=time).rename_axis(f"{area_name}:{today}")
    
    return area_my_df

## 각 지역에 대한 정보를 변수에 저장

In [13]:
서울 = information(서울특별시)
경기 = information(경기도)
인천 = information(인천광역시)
대전 = information(대전광역시)
세종 = information(세종특별자치시)
광주 = information(광주광역시)
대구 = information(대구광역시)
울산 = information(울산광역시)
부산 = information(부산광역시)
제주 = information(제주특별자치도)
강원 = information(강원도)
충북 = information(충청북도)
충남 = information(충청남도)
경북 = information(경상북도)
경남 = information(경상남도)
전북 = information(전라북도)
전남 = information(전라남도)

In [14]:
서울

,누적발전량,일사량
서울특별시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,2.33,1.4
8시,205.01,6.3


In [15]:
경기

,누적발전량,일사량
경기도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,2.33,1.4
8시,205.01,6.3


In [16]:
인천

,누적발전량,일사량
인천광역시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,2.33,1.4
8시,0.29,18.7


In [17]:
대전

,누적발전량,일사량
대전광역시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,1.2
8시,3.17,15.4


In [18]:
세종

,누적발전량,일사량
세종특별자치시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,0.4
8시,0.56,21.7


In [19]:
광주

,누적발전량,일사량
광주광역시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,5.3
8시,1.22,77.5


In [20]:
대구

,누적발전량,일사량
대구광역시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,5.3
8시,0.99,79.7


In [21]:
울산

,누적발전량,일사량
울산광역시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,5.3
8시,4.62,116.3


In [22]:
부산

,누적발전량,일사량
부산광역시:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,5.7
8시,2.24,76.8


In [23]:
제주

,누적발전량,일사량
제주특별자치도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.26,17.9
8시,1.90,139.3


In [24]:
강원

,누적발전량,일사량
강원도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,1.45,20.7
8시,11.44,171.0


In [25]:
충북

,누적발전량,일사량
충청북도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,5.0
8시,8.96,81.2


In [26]:
충남

,누적발전량,일사량
충청남도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.02,1.4
8시,7.81,6.3


In [27]:
경북

,누적발전량,일사량
경상북도:오늘(10/09),,
0시,-,-
1시,-,-
2시,-,-
3시,-,-
4시,-,-
5시,-,-
6시,-,-
7시,-,-
8시,-,-


In [28]:
경남

,누적발전량,일사량
경상남도:오늘(10/09),,
0시,-,-
1시,-,-
2시,-,-
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.00,6.0
8시,20.54,66.5


In [29]:
전북

,누적발전량,일사량
전라북도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.15,5.1
8시,13.43,85.0


In [30]:
전남

,누적발전량,일사량
전라남도:오늘(10/09),,
0시,0.00,0.0
1시,0.00,0.0
2시,0.00,0.0
3시,0.00,0.0
4시,0.00,0.0
5시,0.00,0.0
6시,0.00,0.0
7시,0.45,7.3
8시,23.22,42.8
